## Recursive Function Calling with Ollama

### Requirements

#### 1. Ollama

Ollama installation instructions per OS (macOS, Linux, Windows) can be found on [their website](https://ollama.com/download). For Linux simply (run cell below if not installed): 

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

#### 2. Python Ollama Library

For that:

In [ ]:
%pip install ollama

#### 3. Pull the model from Ollama

Download the q8 quantized NousHermes-2-Pro-Mistral-7B from Ollama (uploaded by adrienbrault):

In [130]:
!ollama pull interstellarninja/hermes-2-pro-llama-3-8b-tools

python(64497) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 10c52

### Usage

#### 1. Define Tools

In [30]:
import requests
import random
from datetime import datetime
import pytz
import time
import json

def get_weather_forecast(location: str) -> dict[str, str]:
    """Retrieves a simple weather forecast for a given location"""
    url = f"https://wttr.in/{location}?format=%C,%t"
    response = requests.get(url)
    if response.status_code == 200:
        condition, temperature = response.text.strip().split(',')
        return {
            "location": location,
            "forecast": condition,
            "temperature": temperature
        }
    else:
        return {"error": "Unable to fetch weather data"}


def get_stock_price(symbol: str) -> float:
    """Retrieves the stock price for a given symbol"""
    api_key = "your_stock_api_key"
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    return float(data["Global Quote"]["05. price"])

def get_random_number(min_value: int, max_value: int) -> int:
    """Returns a random number between min_value and max_value"""
    return random.randint(min_value, max_value)

def get_current_time(time_zone: str, format: str) -> str:
    """Returns the current time in the specified time zone and format"""
    tz = pytz.timezone(time_zone)
    current_time = datetime.now(tz)
    return current_time.strftime(format)

def speak_to_user(assistant_message) -> str:
    """Opens a text input widget for the user to provide feedback or confirm something"""
    print(assistant_message)
    user_input = input("Please provide your feedback or confirmation: ")
    time.sleep(5)  # Wait for 5 seconds
    return user_input

def get_user_location(accuracy: int) -> str:
    """
    Returns the user's location based on the public IP address and accuracy level.
    
    Parameters:
    accuracy (int): The level of detail for the location information.
        1 - Country only
        2 - City and country
        3 - City, region, and country
    
    Returns:
    str: The location information based on the specified accuracy level or an error message.
    """
    try:
        # Retrieve public IP address
        ip_response = requests.get('https://api.ipify.org?format=json')
        ip_response.raise_for_status()
        ip_address = ip_response.json().get('ip')
        
        # Use public IP to get location data
        location_url = f"http://ip-api.com/json/{ip_address}"
        location_response = requests.get(location_url)
        location_response.raise_for_status()
        data = location_response.json()
        
        if data['status'] == 'fail':
            return f"Error in get_user_location: {data.get('message', 'Unknown error')}"
        
        if accuracy == 1:
            return data.get("country", "Unknown country")
        elif accuracy == 2:
            return f"{data.get('city', 'Unknown city')}, {data.get('country', 'Unknown country')}"
        elif accuracy == 3:
            return f"{data.get('city', 'Unknown city')}, {data.get('regionName', 'Unknown region')}, {data.get('country', 'Unknown country')}"
        else:
            return "Invalid accuracy level. Please specify 1 (Country), 2 (City and Country), or 3 (City, Region, and Country)."
    except requests.RequestException as e:
        return f"Error: {e}"


In [31]:
def test_functions():
    print("Testing get_weather_forecast:")
    try:
        weather = get_weather_forecast("London")
        print(f"Weather in London: {weather}")
    except Exception as e:
        print(f"Error in get_weather_forecast: {str(e)}")

    print("\nTesting get_stock_price:")
    try:
        price = get_stock_price("AAPL")
        print(f"Current price of AAPL: ${price:.2f}")
    except Exception as e:
        print(f"Error in get_stock_price: {str(e)}")

    print("\nTesting get_random_number:")
    try:
        number = get_random_number(2, 42)
        print(f"Random number between 1 and 100: {number}")
    except Exception as e:
        print(f"Error in get_random_number: {str(e)}")

    print("\nTesting get_current_time:")
    try:
        time = get_current_time("America/New_York", "%Y-%m-%d %H:%M:%S")
        print(f"Current time in New York: {time}")
    except Exception as e:
        print(f"Error in get_current_time: {str(e)}")

    print("\nTesting get_user_location:")
    try:
        location = get_user_location(2)
        print(f"Location for user's ip address: {location}")
    except Exception as e:
        print(f"Error in get_user_location: {str(e)}")

In [32]:
test_functions()

Testing get_weather_forecast:
Weather in London: {'location': 'London', 'forecast': 'Cloudy ', 'temperature': '+24°C'}

Testing get_stock_price:
Current price of AAPL: $224.31

Testing get_random_number:
Random number between 1 and 100: 26

Testing get_current_time:
Current time in New York: 2024-07-21 10:52:13

Testing get_user_location:
Location for user's ip address: Shibuya, Japan


In [33]:
speak_to_user(assistant_message="Do you confirm")

Do you confirm


'Yes I confirm!'

In [34]:
from langchain_core.utils.function_calling import convert_to_openai_tool

functions = [
    get_weather_forecast,
    get_stock_price,
    get_random_number,
    get_current_time,
    get_user_location,
    speak_to_user
]

tools = [convert_to_openai_tool(t) for t in functions]

In [35]:
import json


print(json.dumps(tools, indent=2))

[
  {
    "type": "function",
    "function": {
      "name": "get_weather_forecast",
      "description": "Retrieves a simple weather forecast for a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string"
          }
        },
        "required": [
          "location"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_stock_price",
      "description": "Retrieves the stock price for a given symbol",
      "parameters": {
        "type": "object",
        "properties": {
          "symbol": {
            "type": "string"
          }
        },
        "required": [
          "symbol"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_random_number",
      "description": "Returns a random number between min_value and max_value",
      "parameters": {
        "type": "object",
        "properties": {
          "min_v

In [44]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

def run_hermes_tool_inference(messages, tools):
    response = client.chat.completions.create(
        model="interstellarninja/hermes-2-pro-llama-3-8b-tools:latest",
        messages = messages,
        tools=tools
    )

    return response

In [127]:
messages = [
    {"role": "user", "content": "Get the user's location first. Once you have the correct user's location, get current weather forecast. Call the functions one at a time sequentially without commenting or asking for confirmation"}
]

In [128]:
def recursive_tool_calling():
    while True:
        response = run_hermes_tool_inference(messages, tools)

        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        print(f"Assistant Message: {assistant_message}")

        if not assistant_message.tool_calls:
            break

        for tool_call in assistant_message.tool_calls:
            function_call = tool_call.function
            name = function_call.name
            arguments = json.loads(function_call.arguments)
            for function in functions:
                if function.__name__ == name:
                    print(f"Invoking tool call: {name}")
                    result = function(**arguments)
                    result_content = {
                        "name": name,
                        "content": result
                    }
                    messages.append(
                        {
                            "role": "tool",
                            "content": json.dumps(result_content),
                            "tool_call_id": tool_call.id
                        }
                    )
                    print(f"Tool Call Result: {result_content}")
                    break

    response = run_hermes_tool_inference(messages, tools)
    assistant_message = response.choices[0].message
    messages.append(assistant_message)

    return messages
            

In [129]:
messages = recursive_tool_calling()

Assistant Message: ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_75etsjm3', function=Function(arguments='{"accuracy":3}', name='get_user_location'), type='function')])
Invoking tool call: get_user_location
Tool Call Result: {'name': 'get_user_location', 'content': 'Shibuya, Tokyo, Japan'}
Assistant Message: ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_gg63d1c3', function=Function(arguments='{"location":"Shibuya, Tokyo, Japan"}', name='get_weather_forecast'), type='function')])
Invoking tool call: get_weather_forecast
Tool Call Result: {'name': 'get_weather_forecast', 'content': {'location': 'Shibuya, Tokyo, Japan', 'forecast': 'Clear ', 'temperature': '+29°C'}}
Assistant Message: ChatCompletionMessage(content='It is currently clear in Shibuya, Tokyo, and the temperature is 29°C (82.2°F). Enjoy your day!', role='assistant', function_c